In [ ]:
#? 什么样的需要合并
#  什么样的需要分开 知识补给（父节点+兄弟节点+自己选一个最合适的）
# 删除 题目 

In [2]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

llm = ChatOpenAI(
    api_key="sk-EQY6oy2D9Brb2IWxymHTnmeFz5unIyobzHtjDgFs2ZwEPjmY",
    base_url="https://api.chatanywhere.tech/v1",
    model="gpt-4o-mini",
    temperature=0.35,
)
from langchain_core.messages import HumanMessage, SystemMessage
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate
)

In [1]:
prompt = '''-任务-
基于给定的知识树结构，自底向上分析每个子节点是否应合并到其直接父节点。需同时考虑父节点和兄弟节点的内容关系。

-输入格式-
父节点: <节点名称>
子节点列表: [<节点1名称>, <节点2名称>, ...]

-分析步骤-
1. **层级检查**：从最底层子节点（Level值最大）开始分析
2. **内容对比**：
   a. 对比子节点与父节点的内容重叠度
   b. 检查兄弟节点间是否有逻辑并列关系
3. **合并条件**（需满足全部）：
   - 子节点内容可被父节点完整覆盖
   - 兄弟节点无相同层级的必要细分（如无并列分类需求）
   - 无考试题型标注（如【名词解释】）
4. **禁止合并**（满足任意一条）：
   - 子节点包含父节点未提及的关键细节
   - 合并后会导致兄弟节点逻辑混乱（如破坏分类体系）

-输出格式-
[评估结果]
父节点: <节点名称>
子节点: [<节点1>, <节点2>]
决策: <合并/保持独立>
理由: 
1. 内容关系: <父子节点内容重叠描述>
2. 结构影响: <对兄弟节点逻辑的影响>
3. 考试需求: <涉及题型或考点说明>

-示例-
######################
示例1:
[评估结果]
父节点: 学龄教育与成人教育
子节点: [成人教育，学龄教育]
决策: <保持独立>
理由: 
1. 内容关系: 父节点仅提分类，子节点含具体形式（夜大、函授）
2. 结构影响: 合并会导致与兄弟节点"学龄教育"不对称
3. 考试需求: 子节点涉及【名词解释】考点

######################'''

In [15]:
import os
os.environ["NEO4J_URI"] = "neo4j+s://5f494a0d.databases.neo4j.io"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "JPvH9AyoJVJDlbOzUml3qfyA_oWutzcMfQCJI9qRCOY"


from neo4j import GraphDatabase

class Neo4jFetcher:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()

    def get_level2_concepts_with_children(self):
        query = """
        MATCH (parent:主要概念 {level:2})-[:包含]->(child:主要概念)
        RETURN ID(parent) AS parent_id, parent.name AS parent_name, parent.page_content AS parent_pg,
               COLLECT([ID(child), child.name, child.page_content]) AS children
        """
        with self.driver.session() as session:
            result = session.run(query)
            return [((record["parent_id"], record["parent_name"], record["parent_pg"]),
                     [(cid, cname, child_pg) for cid, cname, child_pg in record["children"]])
                    for record in result]

Neo4jFetcher = Neo4jFetcher(
    uri=os.environ["NEO4J_URI"],
    user=os.environ["NEO4J_USERNAME"],
    password=os.environ["NEO4J_PASSWORD"]
)

res = Neo4jFetcher.get_level2_concepts_with_children()
print(res)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated function: `id`.} {position: line: 3, column: 16, offset: 74} for query: '\n        MATCH (parent:主要概念 {level:2})-[:包含]->(child:主要概念)\n        RETURN ID(parent) AS parent_id, parent.name AS parent_name, parent.page_content AS parent_pg,\n               COLLECT([ID(child), child.name, child.page_content]) AS children\n        '
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated function: `id`.} {position: line: 4, column: 25, offset: 185} for query: '\n        MATCH (parent:主要概念 {level:2})-[:包含]->(child:主要概念)\

[((4, '专业能力', '\n教师的专业能力就是教师的教育教学能力，是教师在教育教学活动中形成的顺利完成某项任务的本领。教师的专业能力是教师综合素质最突出的外在表现，也是评价教师专业性的核心因素。'), [(5, '教学技巧', '\n教学技巧是“教学行为专业性”的重要方面，反映了教师运用已有知识或经验来完成教学任务的熟练程度和水平，是在掌握初级教学技能的基础上经过反复练习，使教学活动方式达到自动化的程度。教师的基本教学技巧包括：导入的技巧、强化的技巧、变化刺激的技巧、发问的技巧、分组活动的技巧、教学媒体运用的技巧、沟通与表达的技巧、结束的技巧、补救教学的技巧。'), (6, '教学能力', '\n教学能力是指教师顺利完成教学任务所应具备的直接有效的心理特征，教学能力包括以下方面。\n教学设计能力。教学设计能力指教师在具备基本的专业知识和教学技巧的基础上，能够综合运用这些知识和技巧，根据教学大纲的要求设计出适当的年度或单元教学计划的能力。具体来说，就是掌握和运用教学大纲的能力、掌握和运用教材的能力、制订教学计划的能力和编写教案的能力等。\n教学实施能力。教学实施能力是教师在一般教学情况下有效地实施所设计的教学计划，并能根据实际情况控制教学情境的能力。教学实施能力也是多种具体能力的综合，如选择和运用教学方法的能力、因材施教的能力、课堂教学组织的能力、运用各种教学技巧的能力和教学机智等。\n教学检查评价能力。教学检查评价能力是指教师在教学过程中收集资料，运用各种评价方法了解学生的学习情况，以判定教师是否完成了预定的教学目标，学生是否达到了预定的学习目标，从而根据反馈的信息来补救或改进教学工作的能力。具体可分解为：设定评价目标和评价标准的能力，收集评价资料的能力，选择和运用评价方法和评价工具的能力，分析或解释评价资料与结果的能力以及反馈矫正的能力等。'), (7, '交往能力', '\n教师应具有沟通社交能力，特别是协调社会、家庭等方面对学生的教育影响，要善于与各种人员接触交往，以取得他们的理解和配合。'), (8, '组织与管理能力', '\n现代学校具有严密的组织性，为保证教育教学活动顺利进行，教师须具有多方面的组织管理能力，能够确定符合实际活动的预期目标，拟订周密的工作计划，根据课堂情境的变化，迅速调节和完成计划的能力，机智地处理课堂偶发事件的能力。如班级管理能

In [22]:
def get_decs_from_text(title, text):
    # 从文本中提取描述
    decs_sys_template = """
-目标- 
给定一个标题和与标题相关的文本文档，使用一句话描述文本的内容。

-注意事项-
确保描述简洁明了、不存在指代混乱以及冗余。

-示例-
######################
示例1：
标题：自然成熟理论
文本：自然成熟理论的代表人物是格赛尔。他认为支配儿童发展的因素是成熟和学习，他更着重于成熟对儿童发展的制约，认为成熟与主体的内环境有关，而学习则与客观外环境有关。儿童心理发展是儿童行为或心理形式在环境的影响下按一定顺序出现的过程，这个顺序与成熟关系较大，而与外环境关系较小，外环境只是给发展提供适应的时机而已。格赛尔指出无视成熟的学习和训练是徒劳的，是没有成效的，特别是人的早期发展，对内部成熟的依赖程度大大高于外部环境的学习和影响。他的“双生子爬梯试验”提供了儿童发展中成熟优势的实证，为大家所公认。此实验说明了成熟是某些行为、能力产生的必需条件，是身心发展的一种准备状态。在某些人体机能的生理结构未成熟之前，学习和训练的成效是很差的，甚至是不可能进行的，只有在某一行为出现的机体发育临近或达到成熟状态时，学习和训练才能奏效。
——————————————————————
输出：自然成熟理论由格赛尔提出，强调儿童发展主要受生理成熟驱动，外部环境仅提供适应机会。
######################
"""
    decs_sys_prompt = SystemMessagePromptTemplate.from_template(decs_sys_template)

    decs_human_template="""
    -真实数据- 
    ######################
    标题：{title}
    文本：{input_text} 
    ###################### 
    输出：
    """

    from langchain.chains import LLMChain
    chat_prompt = ChatPromptTemplate.from_messages([decs_sys_prompt, decs_human_template])
    llm = ChatOpenAI(
        api_key="sk-EQY6oy2D9Brb2IWxymHTnmeFz5unIyobzHtjDgFs2ZwEPjmY",
        base_url="https://api.chatanywhere.tech/v1",
        model="gpt-4o-mini",
        temperature=0.35,
    )
    chain = LLMChain(llm=llm, prompt=chat_prompt)
    
    answer = chain.invoke({
        "title": title,
        "input_text": text
        })
    
    return answer['text']

In [41]:
def get_parent_decs_from_text(title, text, son_list):
    # 从文本中提取描述
    decs_sys_template = """
-目标- 
给定一个主要概念（包括名称和具体内容）和若干个子概念（包括名称和具体内容），使用一句话总结主要概念和所有子概念的内容。

-注意事项-
确保描述简洁明了、不存在指代混乱以及冗余。

-示例-
######################
示例1：
主要概念：(原始形态的德育及其主要特征, 原始形态的德育是一种尚处于萌芽状态的德育，其主要特征集中体现在四个方面。)
子概念：
[(德育的目的十分明确, 原始形态的德育以传递“古代氏族的自由、平等和博爱”的观念与原始氏族部落的道德规范及其风俗习惯为主。),
 (德育内容极为简单, 主要内容包括氏族部落的宗教信仰、礼节仪式、劳动纪律、风俗习惯等最基本的社会公共生活规范。),
 (德育的方式与人们的劳动及其生活紧密结合, 原始形态的德育尚未从原始人的劳动及其生活中分化出来，德育往往只能在劳动的过程中通过长者与新生代口耳相传的方式在实践活动中进行。),
 (德育的对象具有平等性, 在原始氏族部族中，所有的青少年儿童均可在劳动与生活的过程中接受相同的道德教育。)]
——————————————————————
输出：
原始形态的德育指萌芽阶段的道德教育形态，其特征包括目的明确（传递氏族平等观念与道德规范）、内容简单（涵盖宗教礼仪与劳动纪律等基础规范）、方式与劳动及生活紧密结合（通过实践口耳相传）、对象平等（全体青少年均可接受教育）。
######################
"""
    decs_sys_prompt = SystemMessagePromptTemplate.from_template(decs_sys_template)

    decs_human_template="""
    -真实数据- 
    ######################
    主要概念：{title}
    子概念：{s_list}
    ###################### 
    输出：
    """

    from langchain.chains import LLMChain
    chat_prompt = ChatPromptTemplate.from_messages([decs_sys_prompt, decs_human_template])
    llm = ChatOpenAI(
        api_key="sk-EQY6oy2D9Brb2IWxymHTnmeFz5unIyobzHtjDgFs2ZwEPjmY",
        base_url="https://api.chatanywhere.tech/v1",
        model="gpt-4o-mini",
        temperature=0.35,
    )
    chain = LLMChain(llm=llm, prompt=chat_prompt)
    
    p_title = title + ':' + text
    s_list = ''
    i = 1
    for son in son_list:
        s_list += str(i) +'.' + son[0] + ':' + son[1] + '\n'
    answer = chain.invoke({
        "title": p_title,
        "s_list": s_list
        })
    
    return answer['text']

In [61]:
from neo4j import GraphDatabase
os.environ["NEO4J_URI"] = "neo4j+s://5f494a0d.databases.neo4j.io"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "JPvH9AyoJVJDlbOzUml3qfyA_oWutzcMfQCJI9qRCOY"
# 初始化 Neo4j driver
driver = GraphDatabase.driver(os.environ["NEO4J_URI"], auth=(os.environ["NEO4J_USERNAME"], os.environ["NEO4J_PASSWORD"]))

# description_list = []

def get_children_from_neo4j(node_id):
    with driver.session() as session:
        query = """
        MATCH (parent {id: $id})-[:包含|包括]->(child)
        RETURN child.id AS id, child.name AS name, child.page_content AS content
        ORDER BY child.id
        """
        result = session.run(query, id=node_id)
        return [dict(r) for r in result]

def generate_description(node_name, content, children=None):
    if children and len(children) > 0:
        return get_parent_decs_from_text(node_name, content, children)
    else:
        if len(content) < 10:
            return content
        return get_decs_from_text(node_name, content)

def process_node(node_id, node_name, content):
    # node_id 不在description_list 内
    found = ''
    for item in description_list:
        if item['id'] == node_id:
            found = item['decs']
            break
    
    if found != '':
        return found
    # 获取子节点
    children = get_children_from_neo4j(node_id)
    child_info = []

    for child in children:
        child_id = child["id"]
        child_name = child["name"]
        child_content = child.get("content") or child.get("page_content", "")
        desc = process_node(child_id, child_name, child_content)
        child_info.append((child_name, desc))

    # 生成当前节点描述
    description = generate_description(node_name, content, child_info)

    description_list.append({
        "id": node_id,
        "decs": description
    })

    print(f"✅ 节点 [{node_name}] 描述生成完成")
    print(f"  描述: {description}")
    return description
# 获取所有 level = 0 的节点作为根节点
def get_level0_nodes():
    with driver.session() as session:
        query = """
        MATCH (n:主要概念)
        WHERE n.level = 0
        RETURN n.id AS id, n.name AS name, n.page_content AS content
        ORDER BY n.id
        """
        result = session.run(query)
        return [dict(r) for r in result]

# 入口执行
for root in get_level0_nodes():
    process_node(root["id"], root["name"], root["content"])


✅ 节点 [促进了教育活动的专业化发展] 描述生成完成
  描述: 学校的出现促进了教育活动的专业化发展，使教育从无序向系统化、连续化转变，提高了教育效率。
✅ 节点 [出现了专门从事教育活动的专业人才] 描述生成完成
  描述: 学校的出现促使有知识的人从体力劳动中分离，形成了教师这一新职业，推动了知识的创新与传播。
✅ 节点 [推动了学科门类知识的形成与发展] 描述生成完成
  描述: 学校的出现促使教师对知识进行归纳分类，推动了学科门类的形成与发展。
✅ 节点 [学校产生的意义] 描述生成完成
  描述: 学校的产生意义在于促进教育活动的专业化发展（使教育从无序向系统化转变）、出现专门从事教育的专业人才（形成教师职业并推动知识传播）、以及推动学科门类知识的形成与发展（促使知识归纳分类）。
✅ 节点 [学校的产生] 描述生成完成
  描述: 学校的产生源于经济发展、政治变革和文化积累，意义在于促进教育专业化、形成教师职业以及推动学科知识的发展。
✅ 节点 [教育形式社会化——教育与生产劳动紧密相连] 描述生成完成
  描述: 教育形式在原始社会与生产劳动紧密相连，教育活动主要通过生活和劳动场所进行。
✅ 节点 [教育权利平等化——教育无阶级性、无等级性] 描述生成完成
  描述: 教育权利平等化强调在原始社会中，所有孩子的照管和教养是平等的，体现了教育的无阶级性和无等级性。
✅ 节点 [教育内容实用化——教育传递生活经验和生产知识] 描述生成完成
  描述: 教育内容在原始社会以生活经验和生产知识为主，体现出明显的生活化和实用化特征。
✅ 节点 [教育方法简单化——口耳相传、言传身教、模仿] 描述生成完成
  描述: 原始社会的教育主要依靠口耳相传、言传身教和模仿等简单方法来传递生活经验和道德品质。
✅ 节点 [原始社会的教育] 描述生成完成
  描述: 原始社会的教育特征包括教育形式与生产劳动紧密相连（教育活动在生活和劳动场所进行）、教育权利平等化（所有孩子的教养是平等的，体现无阶级性和无等级性）、教育内容实用化（主要传递生活经验和生产知识）、以及教育方法简单化（依靠口耳相传、言传身教和模仿等方式）。
✅ 节点 [教育基本上与生产劳动相脱离] 描述生成完成
  描述: 。
✅ 节点 [阶级性] 描述生成完成
  描述: 阶级性指的是统治阶级通过教育特权培养人才

In [69]:
description_list

[{'id': 1, 'decs': '制度的概念包括满足人类生存与发展需求的社会关系及规范系统，涵盖宏观、微观和中观三个层面。'},
 {'id': 4, 'decs': '《教育大辞典》将教育定义为一个国家各类教育机构及其管理体系的整体。'},
 {'id': 5, 'decs': '《中国大百科全书》教育卷对教育制度的阐释包括国家性质决定的教育目的和方针，以及各种教育机构的系统。'},
 {'id': 6, 'decs': '《教育百科辞典》对教育制度的解释包括国家性质下的教育目的和设施总称，以及国家内各种教育机构的体系。'},
 {'id': 3,
  'decs': '教育制度是一个广泛应用的概念，其定义包括《教育大辞典》对国家教育机构及管理体系的整体描述、《中国大百科全书》对教育目的和机构系统的阐释，以及《教育百科辞典》对教育目的、设施和机构体系的解释。'},
 {'id': 8, 'decs': '规则要素指教育制度作为规范系统，旨在规范学校成员的行为与关系，并随着社会变迁而不断调整成文与不成文的规范。'},
 {'id': 9, 'decs': '物质要素是教育制度运作所需的基本设备和资料，包括传统教室设施和现代科技设备。'},
 {'id': 10, 'decs': '组织要素探讨了教育制度与学校组织之间的关系，强调通过组织实施制度以实现教育目标。'},
 {'id': 7,
  'decs': '教育制度的构成要素是由规则要素（规范学校成员行为与关系的成文与不成文规范）、物质要素（教育运作所需的基本设备和资料）和组织要素（教育制度与学校组织关系的实施）相互作用而成的有机整体。'},
 {'id': 2,
  'decs': '教育制度是一个广泛的概念，涵盖国家教育机构及管理体系的定义（包括教育目的和机构系统的阐释）以及由规则要素（规范行为与关系的规范）、物质要素（基本设备和资料）和组织要素（教育制度与学校组织关系的实施）相互作用构成的有机整体。'},
 {'id': 12, 'decs': '学校教育制度是指国家各级学校的体系及规则，包括学校性质、培养目标、入学条件等，班级授课制为其核心。'},
 {'id': 13,
  'decs': '学校教育制度的发展受人的身心发展规律和社会因素的共同制约，主要包括生产力水平、政治经济制度、科技发展和文化

In [71]:
from langchain_community.graphs import Neo4jGraph
graph = Neo4jGraph()

newquery = """
UNWIND $data AS item
MATCH (n:`主要概念` {id: item.id})
SET n.description = item.decs
"""
results = graph.query(newquery, params={"data": description_list})

In [6]:
system_message_prompt = SystemMessagePromptTemplate.from_template(prompt)

human_template="""
-真实数据- 
######################
父节点: {parent_node}
子节点列表: {son_list}
###################### 
输出：
"""

from langchain.chains import LLMChain
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_template])

chain = LLMChain(llm=llm, prompt=chat_prompt)
# with open(r"D:\code\agent_demo\kg\output\1_第一节 教育概述.txt", "r") as f:
#     input = f.read()
#     input = input.replace('{', '{{').replace('}', '}}')

    # answer = chain.invoke({
    #     'parent_node': parent_node,
    #     'son_list': son_list,
    #     })
    # print(answer['text'])
    
    # # 保存
    # with open(r"D:\code\agent_demo\kg\outv2\tmp.txt", "w") as f:
    #     f.write(answer['text'])

C:\Users\pixiu\AppData\Local\Temp\ipykernel_17920\3803985122.py:15: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=chat_prompt)


In [20]:
for (pid, pname, parent_pg), children in res:
    # print(f"父节点: {pname}（ID: {pid}）")
    parent_node = pname + parent_pg
    son_list = '['
    for cid, cname, child_pg in children:
        son_list += str(cid) + '\n' + cname + '\n' + child_pg + '\n'
    son_list += ']'
    # print(f"子节点: {son_list}")
    
    answer = chain.invoke({
        'parent_node': parent_node,
        'son_list': son_list,
        })
    print(answer['text'])
    if '<合并>' in answer['text']:
        with open(r"D:\code\agent_demo\kg\outv2\res.txt", "a") as f:
            f.write(f"父节点: {pname}（ID: {pid}）\n")
            f.write(answer['text'])

[评估结果]
父节点: 专业能力
子节点: [教学技巧, 教学能力, 交往能力, 组织与管理能力, 自我反思与教育研究能力]
决策: 保持独立
理由: 
1. 内容关系: 子节点各自涵盖教师专业能力的不同方面，且父节点的定义较为宽泛，无法完全覆盖子节点的具体内容。
2. 结构影响: 合并会导致各子节点间的逻辑关系混乱，无法清晰区分不同能力的具体要求和应用场景。
3. 考试需求: 各子节点涉及不同的能力考点，合并可能导致考点不明确，影响学习和评估的有效性。
[评估结果]
父节点: 心理素质
子节点: [积极乐观的情绪，豁达开朗的心胸，坚忍不拔的毅力]
决策: <保持独立>
理由: 
1. 内容关系: 父节点描述的是心理素质的整体概念，而子节点则分别阐述了具体的心理素质特征，彼此间存在细节差异。
2. 结构影响: 合并后会导致对各个心理素质特征的描述不够清晰，影响理解与记忆。
3. 考试需求: 子节点内容可能涉及具体的考点，保持独立有助于更好地进行分类与复习。
[评估结果]
父节点: 教师群体专业化
子节点: [教育知识技能的体系化，教师教育的专业化，教师资格的制度化，教师活动的团体化]
决策: <保持独立>
理由: 
1. 内容关系: 子节点各自涵盖教师群体专业化的不同方面，内容并不完全重叠，且每个子节点都有其独特的侧重点。
2. 结构影响: 合并会导致对教师群体专业化的理解模糊，影响各个方面的独立性和逻辑清晰性。
3. 考试需求: 子节点涉及不同的专业知识和制度，可能在考试中作为独立考点出现。
[评估结果]
父节点: 叶澜、白益民等人对教师专业发展阶段的划分
子节点: [关注生存阶段, 关注情境阶段, 关注学生阶段]
决策: <保持独立>
理由: 
1. 内容关系: 子节点描述了教师专业发展的不同阶段，涵盖了教师在各个阶段的关注点和特点，父节点提供了一个总体框架，二者内容并不完全重叠。
2. 结构影响: 合并会导致对教师专业发展阶段的理解不清晰，破坏了各阶段的独立性和逻辑关系。
3. 考试需求: 子节点涉及具体的教师发展阶段，可能会成为考点，尤其是在教育学相关的考试中。
[评估结果]
父节点: 初任教师继续发展的前提——教师的上岗培训
子节点: [使初任者明确教师的职责——教书育人，引导新老教师的交流，必须注重对初任教师专业技能的培训和专业发展意识与能力的养成]
决

KeyboardInterrupt: 